In [1]:
include(Pkg.dir("Dyn3d")*"/src/Dyn3d.jl")
using Dyn3d

# choose config files from folder config_files
# include("../src/config_files/2dLink.jl")
# include("../src/config_files/2dFall.jl")
# include("../src/config_files/2dSwim.jl")
# include("../src/config_files/3dHinge.jl")
# include("../src/config_files/3dPrismatic.jl")
# include("../src/config_files/3dCylindrical.jl")
include("../src/config_files/2dThrow.jl")

Config info set up.


In [2]:
# add bodys
bodys = Vector{SingleBody}(nbody) # body system
for i = 1:nbody
    bodys[i] = AddBody(i, config_body) # add body
end

# add joints
joints = Vector{SingleJoint}(njoint) # joint system
for i = 1:njoint
    joints[i] = AddJoint(i, config_joints[i]) # add joint
end

# assemble system to a chain
system = System(ndim, nbody, njoint, gravity, num_params)
bodys, joints, system = AssembleSystem!(bodys, joints, system)
system

ndim = 2, njoint = 4, nbody = 4
ndof = 24, nudof = 10, ncdof = 14, np = 6, na = 4
udof = [3, 9, 10, 11, 15, 16, 17, 21, 22, 23]
udof_p = [9, 10, 15, 16, 21, 22]
udof_a = [3, 11, 17, 23]
nudof_HERK = 6, ncdof_HERK = 18
udof_HERK = [9, 10, 15, 16, 21, 22]
gravity = [0.0, 0.0, 0.0]
kinmap = [1 1; 2 3; 3 3; 4 3]


In [3]:
# # test function UpdatePosition!
# bodys, joints, system = UpdatePosition!(bodys, joints, system)

# # test function UpdateVelocity!
# for i = 1:nbody
#     @assert length(bodys[i].v) == 6
#     @assert length(bodys[i].Xp_to_b) == 36
#     @assert length(joints[i].vJ) == 6
# end
# v = ones(Float64,24)
# bodys, joints, system, vJ = UpdateVelocity!(bodys, joints, system, v)

In [4]:
# init system
bodys, joints, system, soln = InitSystem!(bodys, joints, system, scheme)

# init soln structure
solns = (Soln)[]
push!(solns, soln)

# init VertsHistory struct
vs = []
push!(vs, VertsHistory(system.nbody, bodys))

1-element Array{Any,1}:
 [0.0 0.0 0.176777 0.176777; 0.176777 0.176777 0.353553 0.353553; 0.353553 0.353553 0.53033 0.53033; 0.53033 0.53033 0.707107 0.707107]

[0.0 0.0 0.176777 0.176777; 0.176777 0.176777 0.353553 0.353553; 0.353553 0.353553 0.53033 0.53033; 0.53033 0.53033 0.707107 0.707107]

[0.0 1.0 1.0 0.0; 0.0 1.0 1.0 0.0; 0.0 1.0 1.0 0.0; 0.0 1.0 1.0 0.0]

In [5]:
# advance in time
idx = 0
@time begin
while soln.t < tf
    # advance one step
    soln, bodys, joints, system = HERK!(soln, bodys, joints, system, scheme, tol)
        
    # record soln and verts_i info
    push!(solns, soln)
    push!(vs, VertsHistory(system.nbody, bodys))
        
    # print progress
    idx += 1
    if mod(idx,500) == 1 
        @printf("itr = %d, t = %.3f, dt = %e\n", idx, soln.t, soln.dt)
#         println("center of mass at", MassCenter(bodys, system))
    end
end
@printf("itr = %d, t = %.3f, dt = %e\n", idx, soln.t, soln.dt)
end

itr = 1, t = 0.001, dt = 4.225926e-05
itr = 501, t = 0.064, dt = 1.496888e-04
itr = 1001, t = 0.138, dt = 1.495684e-04
itr = 1501, t = 0.213, dt = 1.484856e-04
itr = 2001, t = 0.287, dt = 1.481534e-04
itr = 2501, t = 0.361, dt = 1.499150e-04
itr = 3001, t = 0.436, dt = 1.520578e-04
itr = 3501, t = 0.512, dt = 1.519191e-04
itr = 4001, t = 0.588, dt = 1.496733e-04
itr = 4501, t = 0.662, dt = 1.482480e-04
itr = 5001, t = 0.737, dt = 1.488428e-04
itr = 5501, t = 0.812, dt = 1.510315e-04
itr = 6001, t = 0.888, dt = 1.538967e-04
itr = 6501, t = 0.966, dt = 1.550201e-04
itr = 7001, t = 1.043, dt = 1.533055e-04
itr = 7501, t = 1.119, dt = 1.509958e-04
itr = 8001, t = 1.194, dt = 1.500859e-04
itr = 8501, t = 1.269, dt = 1.509877e-04
itr = 9001, t = 1.345, dt = 1.535425e-04
itr = 9501, t = 1.422, dt = 1.558271e-04
itr = 10001, t = 1.500, dt = 1.551581e-04
itr = 10501, t = 1.576, dt = 1.516671e-04
itr = 11001, t = 1.652, dt = 1.490594e-04
itr = 11501, t = 1.727, dt = 1.491879e-04
itr = 12001, t =

In [6]:
using MAT
using Interpolations

# create regular time grid and acquire solutions on it
qJ_regs = Float64[]
t_reg = linspace(0,solns[end].t,length(solns))
for i = 1:system.ndof
    t_temp = ([solns[k].t for k = 1:length(solns)],)
    qJ_temp = [solns[k].qJ[i] for k = 1:length(solns)]   
    qJ_reg = interpolate(t_temp, qJ_temp, Gridded(Linear()))[t_reg]
    append!(qJ_regs, qJ_reg)
end
qJ_regs = reshape(qJ_regs,(length(solns), system.ndof))

# get verts info based on this regular grid solution
vs_reg = []
bodys_reg = deepcopy(bodys)
joints_reg = deepcopy(joints)
system_reg = deepcopy(system)
for i = 1:length(solns)
    bodys_reg, joints_reg, system_reg = UpdatePosition!(bodys_reg, joints_reg, system_reg, solns[i].qJ)
    push!(vs_reg, VertsHistory(system.nbody, bodys_reg))
end

# write to .mat file for animation
matwrite("../matlab_plot/verts_i.mat", Dict(
    "ndim" => system.ndim,
    "nbody" => system.nbody,
    "nverts" => bodys[1].nverts,
    "t" => collect(t_reg),
    "verts" => vs_reg
))

In [7]:
# # results printed
# solns[end].qJ
# solns[end].v
# solns[end].v̇
# solns[end].λ
# solns[end].t

# # code analysis
# using BenchmarkTools, Compat
# @benchmark HERK!(soln, bodys, joints, system, scheme, tol)
# @profile HERK!(soln, bodys, joints, system, scheme, tol)
# Profile.print()

# using Coverage
# m = analyze_malloc("../src")
# m[end-20:end]